In [ ]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
!wget -q -O 'toronto.json' https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/a083c865-6d60-4d1d-b6c6-b0c8a85f9c15?format=geojson&projection=4326

In [ ]:
!wget -q -O 'toronto.csv' https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/a083c865-6d60-4d1d-b6c6-b0c8a85f9c15?format=csv&projection=4326

In [ ]:
toronto_data=pd.read_csv('toronto.csv')

In [ ]:
with open('toronto.json') as json_data:
  torontojson=json.load(json_data)

In [ ]:
torontojson=torontojson['features']

In [ ]:
torontojson[0]

{'geometry': {'coordinates': [[[-79.41469317817781, 43.6739104164259],
    [-79.41484930122832, 43.6743388247927],
    [-79.4155279126094, 43.67606998537741],
    [-79.4157867581137, 43.6767302521386],
    [-79.4164385645546, 43.6783329020511],
    [-79.4165782543862, 43.6787785512344],
    [-79.4167583944682, 43.6792484522149],
    [-79.41698203537291, 43.6798247440047],
    [-79.4170902471251, 43.6801047004014],
    [-79.4174797570886, 43.681051830379],
    [-79.4176666295158, 43.6815699096212],
    [-79.4176976020857, 43.68163169892501],
    [-79.4178116560388, 43.6819252777403],
    [-79.4181213711598, 43.6827105222857],
    [-79.4182734938897, 43.6830961662335],
    [-79.418301440907, 43.683167503583],
    [-79.4184313411692, 43.6834966841996],
    [-79.4184870757371, 43.6836294842548],
    [-79.4185410795905, 43.6837582054538],
    [-79.4184092922561, 43.683786393574],
    [-79.4182955283344, 43.6834937157887],
    [-79.4154265333151, 43.684065660547],
    [-79.4135200432832, 43.

In [ ]:
torontojson[0]['geometry']

{'coordinates': [[[-79.41469317817781, 43.6739104164259],
   [-79.41484930122832, 43.6743388247927],
   [-79.4155279126094, 43.67606998537741],
   [-79.4157867581137, 43.6767302521386],
   [-79.4164385645546, 43.6783329020511],
   [-79.4165782543862, 43.6787785512344],
   [-79.4167583944682, 43.6792484522149],
   [-79.41698203537291, 43.6798247440047],
   [-79.4170902471251, 43.6801047004014],
   [-79.4174797570886, 43.681051830379],
   [-79.4176666295158, 43.6815699096212],
   [-79.4176976020857, 43.68163169892501],
   [-79.4178116560388, 43.6819252777403],
   [-79.4181213711598, 43.6827105222857],
   [-79.4182734938897, 43.6830961662335],
   [-79.418301440907, 43.683167503583],
   [-79.4184313411692, 43.6834966841996],
   [-79.4184870757371, 43.6836294842548],
   [-79.4185410795905, 43.6837582054538],
   [-79.4184092922561, 43.683786393574],
   [-79.4182955283344, 43.6834937157887],
   [-79.4154265333151, 43.684065660547],
   [-79.4135200432832, 43.6844456915062],
   [-79.41340878615

In [ ]:
def get_midpoint(cords):
  lat=[]
  lon=[]
  for i in cords[0]:
    lon.append(i[0])
    lat.append(i[1])
  lat = np.mean(lat)
  lon = np.mean(lon)

  return lat,lon

In [ ]:
toronto_data.shape

(140, 18)

In [ ]:
for i in range(140):
  cords= torontojson[i]['geometry']['coordinates']
  lat,lon=get_midpoint(cords)
  toronto_data['LATITUDE'][i]=lat
  toronto_data['LONGITUDE'][i]=lon


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
toronto_data.head(3)

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry,CLASSIFICATION,CLASSIFICATION_CODE
0,11481,2480141,26005521,NaN,96,96,Casa Loma (96),Casa Loma (96),NaN,NaN,-79.406011,43.680555,17545105,3.678385e+06,8214.176485,"{u'type': u'Polygon', u'coordinates': (((-79.4...",NaN,NaN
1,11482,2480140,26005520,NaN,95,95,Annex (95),Annex (95),NaN,NaN,-79.402468,43.673365,17545121,5.337192e+06,10513.883143,"{u'type': u'Polygon', u'coordinates': (((-79.3...",NaN,NaN
2,11483,2480139,26005519,NaN,109,109,Caledonia-Fairbank (109),Caledonia-Fairbank (109),NaN,NaN,-79.459095,43.688206,17545137,2.955857e+06,6849.911724,"{u'type': u'Polygon', u'coordinates': (((-79.4...",NaN,NaN


In [ ]:
toronto_data.columns

Index(['_id', 'AREA_ID', 'AREA_ATTR_ID', 'PARENT_AREA_ID', 'AREA_SHORT_CODE',
       'AREA_LONG_CODE', 'AREA_NAME', 'AREA_DESC', 'X', 'Y', 'LONGITUDE',
       'LATITUDE', 'OBJECTID', 'Shape__Area', 'Shape__Length', 'geometry',
       'CLASSIFICATION', 'CLASSIFICATION_CODE'],
      dtype='object')

In [ ]:
toronto_data = toronto_data[['AREA_NAME','LATITUDE','LONGITUDE']]

In [ ]:
toronto_data.isnull().count()

AREA_NAME    140
LATITUDE     140
LONGITUDE    140
dtype: int64

In [ ]:
toronto_data

,AREA_NAME,LATITUDE,LONGITUDE
0,Casa Loma (96),43.680555,-79.406011
1,Annex (95),43.673365,-79.402468
2,Caledonia-Fairbank (109),43.688206,-79.459095
3,Woodbine Corridor (64),43.675813,-79.314969
4,Lawrence Park South (103),43.714161,-79.407022
...,...,...,...
135,Dorset Park (126),43.762317,-79.280578
136,Centennial Scarborough (133),43.776338,-79.148292
137,Humbermede (22),43.741472,-79.551924
138,Willowdale West (37),43.771438,-79.424984


In [ ]:
city='Toronto, Canada'
geolocator=Nominatim(user_agent='TorontoExplorer')
corrdinates=geolocator.geocode(city)
print(f'Geo-Corrdinates of {city} are {corrdinates.latitude} , {corrdinates.longitude} ')

Geo-Corrdinates of Toronto, Canada are 43.6534817 , -79.3839347 


In [ ]:
map_toronto = folium.Map(location=[corrdinates.latitude,corrdinates.longitude],zoom_start=10)
for an,lat,lon in zip(toronto_data['AREA_NAME'],toronto_data['LATITUDE'],toronto_data['LONGITUDE']):
  folium.CircleMarker(location=[lat,lon],popup=an,fill=True,radius=5).add_to(map_toronto)


In [ ]:
 map_toronto

In [ ]:
CLIENT_ID = 'LYSEAJOEVNXJ5TLJKRTJCRF3ALCZ4JVIU2KEBYROPM5SBRV3' # your Foursquare ID
CLIENT_SECRET = 'ITZDE0R3RABUN1WRFEJEMYK3TD0YPDZHHKXUHAH0XH4FWBDK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LYSEAJOEVNXJ5TLJKRTJCRF3ALCZ4JVIU2KEBYROPM5SBRV3
CLIENT_SECRET:ITZDE0R3RABUN1WRFEJEMYK3TD0YPDZHHKXUHAH0XH4FWBDK


In [ ]:
lat=toronto_data.loc[0,'LATITUDE']
lon=toronto_data.loc[0,'LONGITUDE']

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore'
params = dict(
      client_id=CLIENT_ID,
      client_secret=CLIENT_SECRET,
      v='20180323',
      ll=f'{lat},{lon}',
      limit=100,
      radius=500
      )
results=requests.get(url,params=params).json()

In [ ]:
def get_category(row):
  try:
    category_list=row['categories']
  except:
    category_list=row['venue.categories']
  
  if category_list==0:
    return None
  else:
    return category_list[0]['name']

In [ ]:
v=results['response']['groups'][0]['items']
testvenue=json_normalize(v)

filteredcategories=['venue.name','venue.categories','venue.location.address','venue.location.lat','venue.location.lng']
testvenue=testvenue.loc[:,filteredcategories]
testvenue['venue.categories']=testvenue.apply(get_category,axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [ ]:
columns=['Venue_Name','Categories','Address','Latitude','Longitude']
testvenue.columns=columns

In [ ]:
testvenue.head()


,Venue_Name,Categories,Address,Latitude,Longitude
0,Casa Loma,Castle,1 Austin Terrace,43.677934,-79.409521
1,Baldwin Steps,Historic Site,Davenport Rd.,43.677707,-79.408209
2,Casa Loma Stables,Museum,328 Walmer Rd,43.679395,-79.410905
3,Flor de Sal,Modern European Restaurant,501 Davenport Road,43.677757,-79.407176
4,Sir Winston Churchill Park,Park,301 St Clair Ave W,43.683732,-79.409881


In [ ]:
for i in range(testvenue.last_valid_index()):
  print(testvenue.loc[i,'Categories'])

Castle
Historic Site
Museum
Modern European Restaurant
Park
History Museum
History Museum
Steakhouse
Park
Park
Park
History Museum
Park
Bistro
Café


In [ ]:
toronto_data['AREA_NAME']=pd.DataFrame(toronto_data.AREA_NAME.str.replace('[^a-zA-Z]', ''))

In [ ]:
toronto_data.head()

,AREA_NAME,LATITUDE,LONGITUDE
0,CasaLoma,43.680555,-79.406011
1,Annex,43.673365,-79.402468
2,CaledoniaFairbank,43.688206,-79.459095
3,WoodbineCorridor,43.675813,-79.314969
4,LawrenceParkSouth,43.714161,-79.407022


In [ ]:
def GetVenues(names,latitude,longitude):
  venues=[]
  for an,lat,lon in zip(names,latitude,longitude):
    print(an)
    url = 'https://api.foursquare.com/v2/venues/explore'
    params = dict(
      client_id=CLIENT_ID,
      client_secret=CLIENT_SECRET,
      v='20180323',
      ll=f'{lat},{lon}',
      limit=100,
      radius=500
      )
    results=requests.get(url,params=params).json()['response']['groups'][0]['items']

    try:
      venues.append([(
          an,
          lat,
          lon,
          v['venue']['name'],
          v['venue']['location']['lat'],
          v['venue']['location']['lng'],
          v['venue']['categories'][0]['name']) for v in results
          ])
    except:
      exception.append(an)
      print('***Here***')
      pass
   
  nearby_venues=pd.DataFrame([item for venues in venues for item in venues])
  nearby_venues.columns=['Neighborhood', 'Neighborhood_Latitude', 'Neighborhood_Longitude', 'Venue','Venue_Latitude', 'Venue_Longitude','Venue_Category']
  print("success!")
  return nearby_venues

In [ ]:
toronto_venues=GetVenues(toronto_data['AREA_NAME'],toronto_data['LATITUDE'],toronto_data['LONGITUDE'])

CasaLoma
Annex
CaledoniaFairbank
WoodbineCorridor
LawrenceParkSouth
Milliken
HenryFarm
DownsviewRodingCFB
KingsviewVillageTheWestway
KennedyPark
PleasantView
Steeles
DufferinGrove
BrookhavenAmesbury
HumberSummit
RosedaleMoorePark
YongeEglinton
Oakridge
NorthStJamesTown
HighlandCreek
HighParkSwansea
DonValleyVillage
MountPleasantWest
WestminsterBranson
WestHumberClairville
WestHill
VictoriaVillage
StAndrewWindfields
SouthRiverdale
SouthParkdale
ScarboroughVillage
BroadviewNorth
Guildwood
GreenwoodCoxwell
MountOliveSilverstoneJamestown
BayStreetCorridor
BathurstManor
BanburyDonMills
LAmoreaux
TrinityBellwoods
WestonPellamPark
GlenfieldJaneHeights
Weston
EringateCentennialWestDeane
EnglemountLawrence
BridlePathSunnybrookYorkMills
RockcliffeSmythe
DanforthEastYork
ElmsOldRexdale
EglintonEast
WexfordMaryvale
Alderwood
AgincourtSouthMalvernWest
LittlePortugal
TaylorMassey
HumewoodCedarvale
AgincourtNorth
JunctionArea
MountDennis
EdenbridgeHumberValley
Rustic
BlakeJones
ForestHillNorth
Hillcr

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

(1778, 7)


,Neighborhood,Neighborhood_Latitude,Neighborhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,CasaLoma,43.680555,-79.406011,Casa Loma,43.677934,-79.409521,Castle
1,CasaLoma,43.680555,-79.406011,Baldwin Steps,43.677707,-79.408209,Historic Site
2,CasaLoma,43.680555,-79.406011,Casa Loma Stables,43.679395,-79.410905,Museum
3,CasaLoma,43.680555,-79.406011,Flor de Sal,43.677757,-79.407176,Modern European Restaurant
4,CasaLoma,43.680555,-79.406011,Sir Winston Churchill Park,43.683732,-79.409881,Park


In [ ]:
pd.DataFrame(toronto_venues.groupby('Neighborhood')['Venue'].count()).rename(columns={'Venue':'Count'})

,Count
Neighborhood,
AgincourtNorth,4
AgincourtSouthMalvernWest,21
Alderwood,10
Annex,24
BanburyDonMills,19
...,...
Wychwood,36
YongeEglinton,19
YongeStClair,43


In [ ]:
toronto_onehot=pd.get_dummies(toronto_venues[['Venue_Category']],prefix='',prefix_sep='')
toronto_onehot['Neighbourhood']=toronto_venues['Neighborhood']
column_proper=[toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[column_proper]
toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat Launch,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,...,Soccer Stadium,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,CasaLoma,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,CasaLoma,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,CasaLoma,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,CasaLoma,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,CasaLoma,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
toronto_onehot.shape

(1778, 272)

In [ ]:
toronto_grouping=toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouping.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat Launch,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,...,Soccer Stadium,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,AgincourtNorth,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AgincourtSouthMalvernWest,0.000000,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alderwood,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Annex,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.041667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BanburyDonMills,0.052632,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.052632,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
toronto_grouping.shape

(136, 272)

In [ ]:
for n in toronto_grouping['Neighbourhood']:
  print('---'+n+'---')
  temp=toronto_grouping[toronto_grouping['Neighbourhood']==n].T.reset_index()
  temp.columns=['venue','freq']
  temp=temp.iloc[1:]
  temp['freq'] = temp['freq'].astype(float)
  temp=temp.round({'freq':2})
  print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(5))

---AgincourtNorth---
               venue  freq
0     Ice Cream Shop  0.25
1  Indian Restaurant  0.25
2      Shopping Mall  0.25
3         Beer Store  0.25
4       Neighborhood  0.00
---AgincourtSouthMalvernWest---
                  venue  freq
0    Chinese Restaurant  0.24
1             Pool Hall  0.05
2             BBQ Joint  0.05
3       Motorcycle Shop  0.05
4  Cantonese Restaurant  0.05
---Alderwood---
                 venue  freq
0       Discount Store   0.2
1           Donut Shop   0.1
2  Moroccan Restaurant   0.1
3        Shopping Mall   0.1
4           Bagel Shop   0.1
---Annex---
                venue  freq
0                Café  0.08
1  Italian Restaurant  0.08
2      Sandwich Place  0.08
3                 Pub  0.04
4           BBQ Joint  0.04
---BanburyDonMills---
                     venue  freq
0              Coffee Shop  0.11
1           Clothing Store  0.11
2             Gourmet Shop  0.11
3        Accessories Store  0.05
4  Health & Beauty Service  0.05
---BathurstMano

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def return_most_freq(row,count=10):
  temp=row.iloc[1:]
  temp=temp.sort_values(ascending=False)
  return temp.index.values[0:count]

In [ ]:
indicators=['','st','nd','rd']

columns=['Neighbourhood']
count=10

for i in range(1,count+1):
  try:
    columns.append(f'{i}{indicators[i]} Most_Common_Venue')
  except:
    columns.append(f'{i}th Most_Common_Venue')

Neighbourhood_top_10=pd.DataFrame(columns=columns)
Neighbourhood_top_10['Neighbourhood']=toronto_grouping['Neighbourhood']

for i in range(toronto_grouping.shape[0]):
  Neighbourhood_top_10.iloc[i,1:]=return_most_freq(toronto_grouping.iloc[i,:],count)

In [ ]:
Neighbourhood_top_10.head()

,Neighbourhood,1st Most_Common_Venue,2nd Most_Common_Venue,3rd Most_Common_Venue,4th Most_Common_Venue,5th Most_Common_Venue,6th Most_Common_Venue,7th Most_Common_Venue,8th Most_Common_Venue,9th Most_Common_Venue,10th Most_Common_Venue
0,AgincourtNorth,Ice Cream Shop,Indian Restaurant,Beer Store,Shopping Mall,Zoo,Falafel Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit
1,AgincourtSouthMalvernWest,Chinese Restaurant,Breakfast Spot,Cantonese Restaurant,Bank,Clothing Store,Motorcycle Shop,BBQ Joint,Noodle House,Pool Hall,Seafood Restaurant
2,Alderwood,Discount Store,Donut Shop,Grocery Store,Pizza Place,Convenience Store,Playground,Moroccan Restaurant,Bagel Shop,Shopping Mall,Egyptian Restaurant
3,Annex,Café,Sandwich Place,Italian Restaurant,Pharmacy,BBQ Joint,Metro Station,Burger Joint,Pet Store,Shoe Repair,Liquor Store
4,BanburyDonMills,Coffee Shop,Clothing Store,Gourmet Shop,Accessories Store,Italian Restaurant,Pharmacy,Chocolate Shop,Sandwich Place,Cantonese Restaurant,Middle Eastern Restaurant


In [ ]:

toronto_clustering=toronto_grouping.drop('Neighbourhood',1)

In [ ]:
cluster=8
kmeans=KMeans(init='k-means++',n_clusters=cluster,n_init=15).fit(toronto_clustering)

In [ ]:
kmeans.labels_

array([7, 4, 4, 7, 4, 2, 1, 7, 6, 7, 4, 1, 7, 4, 7, 7, 5, 4, 1, 7, 2, 2,
       7, 7, 7, 7, 6, 7, 7, 1, 4, 4, 4, 7, 7, 7, 2, 7, 7, 2, 1, 1, 1, 4,
       2, 2, 7, 2, 4, 2, 7, 2, 7, 7, 4, 7, 4, 7, 7, 4, 1, 7, 7, 2, 4, 7,
       2, 7, 2, 4, 7, 4, 7, 4, 0, 7, 1, 7, 7, 1, 7, 7, 2, 2, 7, 7, 7, 1,
       4, 1, 7, 2, 7, 7, 4, 7, 2, 2, 7, 3, 2, 1, 2, 4, 7, 0, 7, 6, 7, 4,
       7, 7, 3, 7, 7, 4, 7, 7, 4, 6, 4, 7, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 1, 4], dtype=int32)

In [ ]:
all=list(Neighbourhood_top_10['Neighbourhood'])

In [ ]:
exception=[]
for i in toronto_data['AREA_NAME']:
  if (i not in all):
    exception.append(i)
    print(i)
print('done!\n',exception)
toronto_fixed_data=toronto_data
toronto_fixed_data.set_index('AREA_NAME',inplace=True,drop=True)
for i in exception:
  print(i + 'Deleted')
  toronto_fixed_data.drop(i,inplace=True)

StAndrewWindfields
LongBranch
Humbermede
WillowdaleWest
done!
 ['StAndrewWindfields', 'LongBranch', 'Humbermede', 'WillowdaleWest']
StAndrewWindfieldsDeleted
LongBranchDeleted
HumbermedeDeleted
WillowdaleWestDeleted


In [ ]:
toronto_final=toronto_fixed_data
toronto_final=toronto_final.join(Neighbourhood_top_10.set_index('Neighbourhood'),on='AREA_NAME',)
toronto_final.insert(0,'cluster_labels',kmeans.labels_)
toronto_final.reset_index(inplace=True)
toronto_final.head()

,AREA_NAME,cluster_labels,LATITUDE,LONGITUDE,1st Most_Common_Venue,2nd Most_Common_Venue,3rd Most_Common_Venue,4th Most_Common_Venue,5th Most_Common_Venue,6th Most_Common_Venue,7th Most_Common_Venue,8th Most_Common_Venue,9th Most_Common_Venue,10th Most_Common_Venue
0,CasaLoma,7,43.680555,-79.406011,Park,History Museum,Steakhouse,Tennis Court,Café,Museum,Bistro,Castle,Modern European Restaurant,Historic Site
1,Annex,4,43.673365,-79.402468,Café,Sandwich Place,Italian Restaurant,Pharmacy,BBQ Joint,Metro Station,Burger Joint,Pet Store,Shoe Repair,Liquor Store
2,CaledoniaFairbank,4,43.688206,-79.459095,Park,Women's Store,Cosmetics Shop,Zoo,Falafel Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit
3,WoodbineCorridor,7,43.675813,-79.314969,Café,Brewery,Sandwich Place,Discount Store,Grocery Store,Gym,Restaurant,Diner,Baseball Field,Exhibit
4,LawrenceParkSouth,4,43.714161,-79.407022,Playground,Garden,Park,Gym Pool,Zoo,Event Space,Dumpling Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant


In [ ]:
toronto_final['cluster_labels'].value_counts()

7    66
4    26
2    21
1    14
6     4
3     2
0     2
5     1
Name: cluster_labels, dtype: int64

In [ ]:
map_clusters = folium.Map(location=[corrdinates.latitude,corrdinates.longitude], zoom_start=11)

x = np.arange(count)
ys = [i + x + (i*x)**2 for i in range(count)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_final['LATITUDE'], toronto_final['LONGITUDE'], toronto_final['AREA_NAME'], toronto_final['cluster_labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters